In [1]:
from pyspark.sql import SparkSession

def init_spark():
    spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
    sc = spark.sparkContext
    return spark,sc

spark, sc = init_spark()


In [6]:
staticDataFrame = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("data/retail-data/by-day/*.csv")
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [8]:
# let’s add a total cost column and see on what days a customer spent the most.
from pyspark.sql.functions import window, column, desc, col
from pyspark.sql.functions import desc

staticDataFrame\
.selectExpr(
"CustomerId",
"(UnitPrice * Quantity) as total_cost" ,
"InvoiceDate" )\
.groupBy(
col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")\
.sort(desc("destination_total"))\
.show(5)

AnalysisException: cannot resolve '`destination_total`' given input columns: [CustomerId, sum(total_cost), window];;
'Sort ['destination_total DESC NULLS LAST], true
+- Aggregate [CustomerId#87, window#148], [CustomerId#87, window#148 AS window#142, sum(total_cost#138) AS sum(total_cost)#147]
   +- Filter isnotnull(cast(InvoiceDate#85 as timestamp))
      +- Project [named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(cast(InvoiceDate#85 as timestamp), TimestampType, LongType) - 0) as double) / cast(86400000000 as double))) as double) = (cast((precisetimestampconversion(cast(InvoiceDate#85 as timestamp), TimestampType, LongType) - 0) as double) / cast(86400000000 as double))) THEN (CEIL((cast((precisetimestampconversion(cast(InvoiceDate#85 as timestamp), TimestampType, LongType) - 0) as double) / cast(86400000000 as double))) + cast(1 as bigint)) ELSE CEIL((cast((precisetimestampconversion(cast(InvoiceDate#85 as timestamp), Timestam